In [152]:
import multiprocessing
import pandas as pd
import numpy as np
import networkx as nx
import preprocessing as prep
import sys
import datetime

from multiprocessing import Pool
from bb_binary import FrameContainer, Repository, load_frame_container
from collections import namedtuple 
from pandas import DataFrame, Series

In [153]:
path = "/mnt/data/2015082215/"
conf = .95
dist = 160
ilen = 3
cpus = 6

In [154]:
repo = Repository(path)

In [155]:
number_hours = 1*60*60 #number of hours in total in seconds
slice_len = 10*60   #number of seconds per slice

In [156]:
m="08"
d="22"
h="15"
begin = "2015-{}-{}T{}:00:00Z".format(m,d,h) # %Y-%m-%dT%H:%M:%SZ
begin_ts = datetime.datetime.timestamp(datetime.datetime.strptime(begin, "%Y-%m-%dT%H:%M:%SZ"))

In [157]:
parts = int(number_hours/slice_len)

In [159]:
parts*10

60

In [160]:
tasks = []

for i in list(range(parts)):
    b = begin_ts + (i * slice_len)
    e = (b-0.1) + (slice_len)
    #print("{}-{}".format(datetime.datetime.fromtimestamp(b),datetime.datetime.fromtimestamp(e)))
    tasks.append((b, e))

In [161]:
len(tasks)

6

In [162]:
xmax = 3000
offset = 200

In [163]:
Detection = namedtuple('Detection',
                       ['idx', 'xpos', 'ypos', 'radius', 'zRotation', 'decodedId', 'frame_idx', 'timestamp', 'cam_id', 'fc_id'])

In [164]:
begin_ts = tasks[1][0]
end_ts = tasks[1][1]

In [165]:
print(begin_ts,end_ts)

1440256200.0 1440256799.9


In [166]:
# one df per cam
parts = np.empty(4, dtype=object)

for i in list(range(4)):
    print(i)

    tpls = []
    myid = 0
    for f, fc in repo.iter_frames(begin=begin_ts, end=end_ts, cam=i):
        for d in f.detectionsUnion.detectionsDP:
            d = Detection(d.idx, d.xpos, d.ypos, d.radius, d.zRotation, list(d.decodedId), myid, f.timestamp, fc.camId, fc.id)
            tpls.append(d)
        myid += 1

    df = DataFrame(tpls)
    df = prep.calcIds(df,conf)
    parts[i] = df

0
1
2
3


In [167]:
# cam 0 und cam1 nach rechts verschieben
parts[0].xpos = parts[0].xpos + xmax + offset
parts[1].xpos = parts[1].xpos + xmax + offset

In [168]:
# Seiten zusammenfugen
side0 = pd.concat([parts[3], parts[0]])
side1 = pd.concat([parts[2], parts[1]])

In [169]:
close1 = prep.get_close_bees_ckd(side0, dist)
close2 = prep.get_close_bees_ckd(side1, dist)

In [170]:
close = pd.concat([close1,close2])

In [171]:
p = prep.bee_pairs_to_timeseries(close)

In [172]:
edges =  prep.extract_interactions(p,ilen)

In [173]:
filename = "testframes"

In [174]:
G = prep.create_graph2(edges)

In [175]:
nx.write_graphml(G, "{}_{}conf_{}dist_{}ilen".format(filename,str(c), str(dist), str(ilen)) + ".graphml")
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 2220
Number of edges: 26680
Average degree:  24.0360


# Ohne Multithreading